In [ ]:
import pandas as pd
import numpy as np

In [ ]:
#Creating fake data for the example
words = ["word " + str(number) for number in np.arange(0,10)] * 2
documents = ["doc 1"] * 10 + ["doc 2"] * 10
classes = ["class 1"] * 10 + ["class 2"] * 10
counts = np.random.randint(1, 20, 20)
# Create dataframe, and deliberately at missing data by select 1:-1 (this drops the first and last row)
df = pd.DataFrame({'document':documents, 'word':words, 'class':classes, 'count': counts}).iloc[1:-1]
df

,class,count,document,word
1,class 1,17,doc 1,word 1
2,class 1,3,doc 1,word 2
3,class 1,10,doc 1,word 3
4,class 1,3,doc 1,word 4
5,class 1,1,doc 1,word 5
6,class 1,10,doc 1,word 6
7,class 1,4,doc 1,word 7
8,class 1,5,doc 1,word 8
9,class 1,7,doc 1,word 9
10,class 2,12,doc 2,word 0


The example DataFrame, above, is a "long" dataframe with each row representing the count for a word for a document. It is expect that there each document/word has one row; if it doesn't, do a `groupby` with a `sum` for the column.

To make it `wide`, here is one example:

In [ ]:
wide_df = df.pivot(index='document', columns='word', values='count')
wide_df

word,word 0,word 1,word 2,word 3,word 4,word 5,word 6,word 7,word 8,word 9
document,,,,,,,,,,
doc 1,NaN,17.0,3.0,10.0,3.0,1.0,10.0,4.0,5.0,7.0
doc 2,12.0,9.0,6.0,2.0,7.0,1.0,4.0,5.0,17.0,NaN


Note that doc1 didn't have word 0, and doc2 didn't have word 9, so they have NaN (Not a Number) values. We can fill these in with fillna(0). Redoing the previous step in a better way:

In [ ]:
wide_df = df.pivot(index='document', columns='word', values='count').fillna(0)
wide_df

word,word 0,word 1,word 2,word 3,word 4,word 5,word 6,word 7,word 8,word 9
document,,,,,,,,,,
doc 1,0.0,17.0,3.0,10.0,3.0,1.0,10.0,4.0,5.0,7.0
doc 2,12.0,9.0,6.0,2.0,7.0,1.0,4.0,5.0,17.0,0.0


In [ ]:
df[['document', 'word', 'count']]

,document,word,count
1,doc 1,word 1,17
2,doc 1,word 2,3
3,doc 1,word 3,10
4,doc 1,word 4,3
5,doc 1,word 5,1
6,doc 1,word 6,10
7,doc 1,word 7,4
8,doc 1,word 8,5
9,doc 1,word 9,7
10,doc 2,word 0,12


In [ ]:
summed_counts = df.groupby(['document', 'word'])[['count']].sum()
summed_counts

count
document word         
doc 1    word 1     17
         word 2      3
         word 3     10
         word 4      3
         word 5      1
         word 6     10
         word 7      4
         word 8      5
         word 9      7
doc 2    word 0     12
         word 1      9
         word 2      6
         word 3      2
         word 4      7
         word 5      1
         word 6      4
         word 7      5
         word 8     17

Note also that we only kept the document information as the index. The class labels are still in the long DataFrame.

Here, I 
 1. select just those two columns
 2. only look at the unique combinations
 3. set the index to document so it mimics `wide_df`. This is optional, but helps consistency.

In [ ]:
labels = (df[['document', 'class']]
            .drop_duplicates()
            .set_index('document')
          )
labels

,class
document,
doc 1,class 1
doc 2,class 2


*Important*: when sending things to SciKit Learn, make sure the rows on the training data and labels are in the same order! Here, they are correct (e.g. doc1 is the first row both times, doc2 is the second row both times).

If they were incorrect, you can take the index from the data (`wide_df.index`) and select the rows in labels to match that order, like this:

In [ ]:
labels.loc[wide_df.index]

,class
document,
doc 1,class 1
doc 2,class 2
